In [1]:
# ./data 디렉토리 생성
!mkdir -p ./data

# images.tar.gz 압축 해제
!tar -xzf images.tar.gz -C ./data

1. !mkdir -p ./data


    - mkdir: 디렉토리를 생성하는 명령어
	- -p: 이 옵션은 부모 디렉토리도 함께 생성하도록 해주는 옵션  

    예를 들어, ./data 디렉토리가 이미 존재하면 오류를 발생시키지 않고 디렉토리가 없으면 생성
	- ./data: 생성할 디렉토리의 경로  
     현재 작업 디렉토리(./)에 data라는 이름의 디렉토리를 생성하겠다는 의미

2. !tar -xzf images.tar.gz -C ./data


    - tar: 파일을 압축하거나 압축을 해제하는 명령어
	- -x: 압축을 해제하는 옵션
	- -z: .gz 확장자를 가진 gzip 형식의 압축 파일을 다루겠다는 옵션
	- -f: 파일 이름을 지정하는 옵션  
    그 뒤에 오는 images.tar.gz는 압축 해제할 파일 이름
	- -C ./data: 압축 해제된 파일을 ./data 디렉토리로 이동시키겠다는 옵션 즉, 현재 작업 디렉토리의 data 폴더에 압축된 파일들을 풀어 놓겠다는 의미




In [2]:
!ls ./data/images | head # 디렉토리 내의 파일 목록 나열, 첫 10개 항목 출력

Abyssinian_100.jpg
Abyssinian_100.mat
Abyssinian_101.jpg
Abyssinian_101.mat
Abyssinian_102.jpg
Abyssinian_102.mat
Abyssinian_103.jpg
Abyssinian_104.jpg
Abyssinian_105.jpg
Abyssinian_106.jpg


## 데이터셋 클래스





In [3]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image

# PetDataset 클래스 정의
class PetDataset(Dataset):
    """
    Oxford-IIIT Pet Dataset용 커스텀 Dataset 클래스
    """
    def __init__(self, root_dir, annotation_file, transform=None):
        """
        Args:
            root_dir (str): 데이터셋의 루트 디렉토리 경로 (예: './data').
            annotation_file (str): 어노테이션 파일 경로 (예: './data/trainval.txt').
            transform (callable, optional): 이미지에 적용할 변환(transform) 함수.
        """
        self.root_dir = root_dir
        self.transform = transform

        # 이미지 디렉토리와 라벨 매칭
        image_dir = os.path.join(root_dir, 'images')
        image_files = set(os.listdir(image_dir))

        # 어노테이션 파일 읽기
        with open(annotation_file, 'r') as f:
            lines = f.readlines()

        # 이미지 이름과 라벨 매칭
        self.data = []
        for line in lines:
            components = line.strip().split()
            image_name = components[0] + '.jpg'
            label = int(components[1]) - 1  # 라벨을 0-based로 변환
            if image_name in image_files:
                self.data.append((image_name, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): 데이터 인덱스

        Returns:
            image (Tensor): 전처리된 이미지
            label (int): 이미지의 클래스 라벨
        """
        image_name, label = self.data[idx]
        image_path = os.path.join(self.root_dir, 'images', image_name)
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Transform 정의
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 이미지 크기 조정
    transforms.ToTensor(),         # Tensor 변환
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
])

# 데이터셋 경로 및 어노테이션 파일 경로
root_dir = './data'
trainval_annotation_file = 'trainval.txt'
test_annotation_file = 'test.txt'

# Train/Validation/Test Dataset 생성
trainval_dataset = PetDataset(root_dir, trainval_annotation_file, transform=transform)
test_dataset = PetDataset(root_dir, test_annotation_file, transform=transform)

# Train/Validation Split
train_size = int(0.8 * len(trainval_dataset))
val_size = len(trainval_dataset) - train_size
train_dataset, val_dataset = random_split(trainval_dataset, [train_size, val_size])

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# 데이터셋 크기 확인
print(f"Train Dataset Size: {len(train_dataset)}")
print(f"Validation Dataset Size: {len(val_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")

# 첫 번째 데이터 확인
image, label = train_dataset[0]
print(f"First Train Image Shape: {image.shape}, Label: {label}")

Train Dataset Size: 2944
Validation Dataset Size: 736
Test Dataset Size: 3669
First Train Image Shape: torch.Size([3, 128, 128]), Label: 31


Oxford-IIIT Pet Dataset을 사용하여 PyTorch에서 학습을 위한 데이터셋을 준비, 해당 데이터셋을 DataLoader로 로드하여 훈련 및 검증을 수행할 수 있게 만드는 과정

1. PetDataset 클래스 (Custom Dataset 클래스)


    - PetDataset 클래스는 torch.utils.data.Dataset을 상속받아 사용자 정의 데이터셋 클래스를 만듭니다. 이 클래스는 이미지 파일과 해당하는 라벨을 매칭하여 반환합니다.

__init__ 메서드:

    - root_dir: 데이터셋의 루트 디렉토리 경로
	- annotation_file: 어노테이션 파일의 경로로, trainval.txt와 test.txt 파일을 사용
	- transform: 이미지에 적용할 변환 함수입니다. 예를 들어, 크기 조정, 텐서 변환, 정규화
	- image_dir: 이미지 파일이 저장된 디렉토리입 (root_dir/images)
	- image_files: 이미지 파일들의 집합을 만들어 모든 이미지를 읽을 수 있도록 함
	- 어노테이션 파일(trainval.txt, test.txt)을 읽어들여 각 이미지와 그에 해당하는 라벨을 매칭
	- self.data: 각 이미지의 경로와 라벨을 튜플 형태로 저장

__len__ 메서드:

    - 데이터셋의 크기를 반환 즉, 데이터셋에 포함된 이미지의 개수를 반환

__getitem__ 메서드:

    - 주어진 인덱스 idx에 대해 이미지와 라벨을 반환
	- 이미지는 PIL 라이브러리로 로드되며, transform이 주어지면 해당 변환을 적용

2. Transform 정의



    - transform은 torchvision.transforms의 기능을 사용하여 이미지에 적용할 전처리 과정을 정의
	- transforms.Resize((128, 128)): 이미지를 128x128 크기로 리사이즈
	- transforms.ToTensor(): 이미지를 텐서로 변환
	- transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]): 이미지를 정규화

3. 데이터셋 경로 및 어노테이션 파일 경로 설정



    - root_dir: 데이터셋이 저장된 루트 디렉토리 이 경로에 이미지와 어노테이션 파일이 존재
	- trainval_annotation_file 및 test_annotation_file: 각각 훈련용/검증용 및 테스트용 이미지에 대한 어노테이션 파일

4. Train/Validation Split


	- trainval_dataset: 훈련 및 검증 데이터셋을 포함하는 객체
	- train_size와 val_size: 훈련용 데이터와 검증용 데이터의 비율을 80%와 20%로 나누어 훈련 데이터셋과 검증 데이터셋을 분할
	- random_split 함수를 사용하여 훈련과 검증 데이터셋을 랜덤하게 분리

5. DataLoader 생성


	- train_loader, val_loader, test_loader: DataLoader 객체는 배치 처리를 지원하며 데이터를 효율적으로 불러오기 위해 사용
	- train_loader: 훈련 데이터셋을 배치 크기 32로 불러옴
	- val_loader: 검증 데이터셋을 배치 크기 32로 불러옴
	- test_loader: 테스트 데이터셋을 배치 크기 32로 불러옴
	- num_workers=2: 데이터를 로드할 때 사용할 프로세스 수

6. 데이터셋 크기 및 샘플 확인



	- print(f"Train Dataset Size: {len(train_dataset)}"): 훈련 데이터셋의 크기를 출력
	- print(f"Validation Dataset Size: {len(val_dataset)}"): 검증 데이터셋의 크기를 출력
	- print(f"Test Dataset Size: {len(test_dataset)}"): 테스트 데이터셋의 크기를 출력
	- image, label = train_dataset[0]: 훈련 데이터셋에서 첫 번째 이미지를 가져옴
	- print(f"First Train Image Shape: {image.shape}, Label: {label}"): 첫 번째 이미지의 형태와 해당 라벨을 출력



In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        # Block 1: Convolutional Layer, BatchNorm, Dropout, Skip Connection
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.3)

        # Skip connection for Block 1
        self.skip_conv1 = nn.Conv2d(3, 64, kernel_size=1, stride=2, padding=0)  # Skip connection with downsampling
        self.skip_bn1 = nn.BatchNorm2d(64)

        # Block 2: Convolutional Layer, BatchNorm, Dropout, Skip Connection
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.4)

        # Skip connection for Block 2
        self.skip_conv2 = nn.Conv2d(64, 128, kernel_size=1, stride=2, padding=0)
        self.skip_bn2 = nn.BatchNorm2d(128)

        # Block 3: Convolutional Layer, BatchNorm, Dropout
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.5)

        # Fully Connected Layer
        self.fc_input_dim = 256 * 16 * 16  # Assuming input image size is 128x128
        self.fc1 = nn.Linear(self.fc_input_dim, 512)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Block 1 with skip connection
        residual = self.skip_bn1(self.skip_conv1(x))  # Downsampled residual
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        x += residual  # Skip connection after pooling

        # Block 2 with skip connection
        residual = self.skip_bn2(self.skip_conv2(x))  # Downsampled residual
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        x += residual  # Skip connection after pooling

        # Block 3 without skip connection
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)
        x = self.fc2(x)

        return x

1. __init__ 메서드 (모델 초기화)

모델의 레이어를 정의하는 부분  
각 레이어는 nn.Conv2d, nn.BatchNorm2d, nn.Dropout 등을 사용하여 초기화


	- conv1, conv2: 첫 번째 합성곱 층 conv1은 입력 채널 수인 3(색상 채널)을 64개의 특징 맵으로 변환하며, conv2는 conv1의 출력인 64개의 채널을 다시 64개로 변환
	- bn1, bn2: 각각 conv1, conv2 뒤에 배치 정규화를 적용하여, 학습을 안정시키고 빠르게 수렴하도록 도움
	- pool1: MaxPool2d를 사용해 2x2 풀링을 하여 공간적인 크기를 절반으로 줄임
	- dropout1: 드롭아웃을 30%로 적용하여 과적합을 방지

Skip Connection:

	- skip_conv1, skip_bn1: conv1 레이어와 conv2 레이어의 입력을 그대로 전달하려는 목적의 스킵 연결 이 연결은 합성곱 연산 후 차원을 맞추기 위해 1x1 커널을 사용하고, 배치 정규화가 적용됨
	- conv3, conv4: 두 번째 합성곱 층 conv3은 64개의 채널을 128개의 특징 맵으로 변환하고, conv4는 conv3의 출력인 128개의 채널을 다시 128개로 변환
	- bn3, bn4: 각각 conv3, conv4 뒤에 배치 정규화를 적용
	- pool2: MaxPool2d로 풀링을 하여 공간 크기를 절반으로 줄임
	- dropout2: 드롭아웃을 40%로 설정하여 과적합을 방지


Skip Connection:

	- skip_conv2, skip_bn2: conv3와 conv4의 출력을 스킵 연결로 전달하기 위한 레이어
	- conv5, conv6: 세 번째 합성곱 층 conv5는 128개의 채널을 256개의 특징 맵으로 변환하고, conv6은 conv5의 출력인 256개의 채널을 다시 256개로 변환
	- bn5, bn6: 각각 conv5, conv6 뒤에 배치 정규화를 적용
	- pool3: 다시 한 번 풀링을 하여 출력 크기를 절반으로 줄임
	- dropout3: 드롭아웃을 50%로 설정하여 과적합을 방지
	- Fully Connected Layer (FC):
	- fc_input_dim: 256 * 16 * 16은 입력 이미지 크기가 128x128일 때, 합성곱 연산 후에 얻어지는 특징 맵의 차원 이를 1D 벡터로 평탄화하여 FC 레이어에 전달함
	- fc1: 첫 번째 완전 연결(fully connected) 층으로 512개의 출력을 가짐
	- dropout4: FC 층에도 드롭아웃을 적용하여 과적합을 방지
	- fc2: 마지막 출력층으로, num_classes개의 출력을 가짐 이는 분류할 클래스의 개수


2. forward 메서드 (순전파)

forward 메서드는 모델에 대한 순전파 과정을 정의  
입력 데이터가 모델을 통해 어떻게 처리되는지 설명

    - 첫 번째  블록 (Block 1):
	    - residual: skip_conv1과 skip_bn1을 사용하여 스킵 연결로 전달할 값을 계산
	    - x = F.relu(self.bn1(self.conv1(x))): conv1에 배치 정규화와 ReLU 활성화 함수를 적용
	    - x += residual: 이전에 계산된 residual을 더하여 스킵 연결을 구현

	- 두 번째 블록 (Block 2):
	    - residual: skip_conv2와 skip_bn2를 사용하여 스킵 연결을 전달할 값을 계산
	    - x = F.relu(self.bn3(self.conv3(x))): conv3에 배치 정규화와 ReLU를
        적용
	    - x += residual: 이전에 계산된 residual을 더하여 스킵 연결을 구현

	- 세 번째 블록 (Block 3):
	    - x = F.relu(self.bn5(self.conv5(x))): conv5에 배치 정규화와 ReLU를 적용 이후 풀링과 드롭아웃을 적용

	- 평탄화 (Flatten):
	    - x.view(x.size(0), -1): 3D 텐서를 1D로 평탄화하여 FC 층에 전달할 수 있게 함

	- 완전 연결 층 (Fully Connected Layers):
	    - x = F.relu(self.fc1(x)): 첫 번째 FC 층을 통과시킴
	    - x = self.fc2(x): 마지막 FC 층을 통과시켜 최종 출력을 얻음

In [ ]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import matplotlib.pyplot as plt
from IPython.display import clear_output

# 하이퍼파라미터 설정
num_classes = 37
num_epochs = 80
learning_rate = 0.001  # AdamW에서 잘 동작하는 학습률로 설정
weight_decay = 1e-4  # L2 규제

# 디바이스 설정 (GPU 또는 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델, 손실 함수, 옵티마이저 설정
model = CustomCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# CosineAnnealingWarmRestarts 스케줄러
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

# 학습과 검증 결과 기록
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# 학습 루프
for epoch in range(num_epochs):
    # Training Phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    # Learning rate 조정 (CosineAnnealing)
    scheduler.step()

    # 학습률 출력 (현재 학습률을 출력)
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Learning Rate: {current_lr:.6f}")

    # 출력 (매 에폭마다 train loss, val loss, train accuracy, val accuracy 출력)
    clear_output(wait=True)  # 이전 출력 삭제

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
          f"Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

    # 실시간 그래프 출력 (Loss, Accuracy)
    plt.figure(figsize=(12, 6))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epoch + 2), train_losses, label='Train Loss', marker='o')
    plt.plot(range(1, epoch + 2), val_losses, label='Validation Loss', marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train and Validation Loss')
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epoch + 2), train_accuracies, label='Train Accuracy', marker='o')
    plt.plot(range(1, epoch + 2), val_accuracies, label='Validation Accuracy', marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Train and Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

1. 하이퍼파라미터 설정


    - num_classes: 데이터셋에서의 클래스 수 (여기서는 37).
	- num_epochs: 모델 학습을 반복할 횟수 여기서는 80 에폭으로 설정
	- learning_rate: 모델 학습의 초기 학습률을 설정 AdamW 옵티마이저에서 잘 동작하는 값으로 설정
	- weight_decay: L2 규제값으로, 모델의 가중치가 과도하게 커지지 않도록 돕는 역할

2. 디바이스 설정


	- device: 모델이 실행될 디바이스를 설정 GPU가 사용 가능한 경우 GPU를, 그렇지 않으면 CPU를 사용

3. 모델, 손실 함수,옵티마이저 설정


	- model: CustomCNN이라는 커스텀 CNN 모델을 정의하여 device에 맞게 전송
	- criterion: 손실 함수로 CrossEntropyLoss를 사용 다중 클래스 분류 문제에서 자주 사용하는 손실 함수
	- optimizer: AdamW 옵티마이저를 사용하며, 학습률과 L2 규제를 설정

4. CosineAnnealingWarmRestarts 스케줄러 설정


	- scheduler: CosineAnnealingWarmRestarts는 학습률을 일정 주기로 코사인 곡선처럼 줄이고, 다시 주기적으로 학습률을 증가시키는 스케줄러
	- T_0=10: 첫 번째 주기의 길이를 10으로 설정
	- T_mult=2: 주기가 끝날 때마다 주기의 길이를 2배로 늘림
	- eta_min=1e-6: 최소 학습률을 1e-6으로 설정

5. 학습 및 검증 기록


	- 학습 중에 기록할 변수들을 초기화
    - 각 에폭마다 train_losses, val_losses, train_accuracies, val_accuracies 리스트에 값을 추가하여 후에 결과를 분석

6. 학습 루프  


	- 학습 모드로 설정: model.train()은 모델을 학습 모드로 설정하여 Dropout이나 BatchNorm과 같은 레이어가 학습 중처럼 동작하도록 함
	- 배치마다 연산: train_loader에서 배치 단위로 이미지를 불러와서, 모델에 입력하고 손실을 계산하여, 역전파(backpropagation) 후 옵티마이저로 업데이트
	- 손실과 정확도 기록: 각 배치의 손실과 정확도를 계산하여 running_loss, correct, total에 누적

7. 검증 루프


	- 평가 모드 설정: model.eval()은 모델을 평가 모드로 설정 이 모드에서는 Dropout과 BatchNorm이 평가 모드로 동작
	- 검증 배치 처리: torch.no_grad()는 gradient 계산을 하지 않도록 하여 테스트에 소모되는 메모리와 시간을 줄임
    - 검증 데이터에 대해 예측을 하고 손실 및 정확도를 계산

8. 학습률 스케줄링


	- 학습률 스케줄러가 각 에폭마다 학습률을 조정하도록 함
    - CosineAnnealingWarmRestarts는 학습률을 일정 주기로 조정하여 모델이 더 잘 수렴할 수 있도록 도움

9. 실시간 그래프 출력


	- Loss: train_losses와 val_losses를 시각화하여 훈련과 검증의 손실을 비교
	- Accuracy: train_accuracies와 val_accuracies를 시각화하여 훈련과 검증 정확도를 비교

In [ ]:
def test_model(model, test_loader, criterion, device):
    model.eval()  # 모델을 평가 모드로 전환
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # 테스트 시에는 gradient 계산을 하지 않음
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()  # 손실값 누적

            # 예측 결과 계산
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # 평균 테스트 손실 및 정확도 계산
    test_loss /= len(test_loader)
    test_accuracy = 100 * correct / total

    # 테스트 결과 출력
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

    return test_loss, test_accuracy

test_model() 함수는 주어진 테스트 데이터셋에 대해 모델을 평가하는 함수  
이 함수는 모델을 평가 모드로 전환하고, 테스트 데이터셋을 사용하여 손실(loss)과 정확도(accuracy)를 계산


1.	model.eval():


	- 이 명령어는 모델을 “평가 모드”로 전환
    - 평가 모드에서는 Dropout 레이어나 Batch Normalization 레이어가 학습 모드와 다르게 동작 - 학습 시에는 랜덤하게 뉴럴 네트워크를 비활성화하는 Dropout과 같은 레이어가 평가 시에는 항상 활성화됨

2.	test_loss = 0.0, correct = 0, total = 0:


	- 테스트 과정에서 사용할 변수들을 초기화
	- test_loss: 테스트 데이터셋의 평균 손실을 계산하기 위한 변수
	- correct: 정확도를 계산하기 위해 맞춘 예측의 수
	- total: 전체 샘플 수

3.	with torch.no_grad()::


	- torch.no_grad()는 그라디언트 계산을 비활성화 이는 테스트 시에는 역전파(backpropagation) 과정이 필요 없기 때문에 메모리 사용과 계산 속도를 줄여줌
    - 이 블록 안에서는 파라미터의 업데이트가 이루어지지 않음

4.	for images, labels in test_loader::


	- test_loader는 테스트 데이터를 배치(batch) 단위로 불러오는 DataLoader  
    - 각 배치마다 images와 labels가 test_loader에서 반환

5.	images, labels = images.to(device), labels.to(device):


	- images와 labels를 설정된 device (GPU 또는 CPU)로 이동시킴
    - GPU에서 모델을 학습시키고 있다면 데이터를 GPU로 옮겨서 처리 속도를 빠르게 함

6.	outputs = model(images):


	- 모델에 이미지를 입력하여 예측된 값을 outputs로 받음

7.	loss = criterion(outputs, labels):


	- 모델의 예측값(outputs)과 실제 레이블(labels)을 비교하여 손실을 계산
    - criterion은 손실 함수로, 보통 CrossEntropyLoss 같은 분류 손실 함수를 사용

8.	test_loss += loss.item():


	- 배치마다 계산된 손실값을 test_loss에 누적
    - .item()은 텐서에서 값을 추출하여 파이썬 숫자 형태로 반환

9.	_, predicted = torch.max(outputs.data, 1):


	- outputs.data는 모델의 예측 결과 torch.max() 함수는 각 배치에 대해 가장 큰 값을 갖는 인덱스를 반환 predicted는 모델이 예측한 클래스 인덱스를 나타냄

10.	total += labels.size(0):


	- 현재 배치에서의 샘플 수를 total에 추가

11.	correct += (predicted == labels).sum().item():


	- 예측한 클래스(predicted)와 실제 레이블(labels)이 일치하는지를 확인하여 맞춘 예측의 개수를 누적

12.	test_loss /= len(test_loader):


	- 전체 배치에 대해 평균 손실을 계산

13.	test_accuracy = 100 * correct / total:


	- 전체 테스트 데이터에서 모델의 정확도를 계산
    - 정확도는 맞춘 예측의 수를 전체 샘플 수로 나누어 백분율로 계산

14.	print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%"):


	- 계산된 테스트 손실과 정확도를 출력
    
15. return test_loss, test_accuracy:


	- 최종 테스트 손실과 정확도를 반환

### 특이사항:   
처음에 데이터셋을 다운받았는데 annotation안에서 어떤 파일을 이용해서 학습 및 테스트를 진행하는지 혼동이 와서 진행이 더딘 점이 있었다  
학습 진행 중 GPU를 다 써서 학습 완료를 하지 못하였음

### 분석, 고찰:   
주석 및 설명

### 어려웠던 점:
모델 구축은 주로 로컬 컴퓨터에서 진행했었고, 코랩을 이용해본 건 거의 처음이었습니다. 로컬에서는 GPU나 연결 문제로 작업이 중단되는 일이 드물지만, 코랩 환경에서는 불편함을 겪었습니다. 모델 구조를 설정하고 코드를 작성하는 과정에서 차원 오류가 많이 발생했고, 이를 해결하는 데 어려움이 있었습니다. 각 레이어에서 계산된 결과를 다음 레이어로 넘길 때 발생하는 오류를 줄이고, 계산이 제대로 이루어지도록 하는 데 추가적인 공부가 필요하다고 느꼈습니다.

하이퍼파라미터 튜닝을 진행했지만, 낮은 에폭수에서는 큰 체감을 얻지 못했으며, 학습을 완료하지 못해 모델 성능의 한계인지, 아니면 튜닝이 부족한 것인지를 명확히 파악하기 어려웠습니다. 모델에 따라, 데이터셋에 따라 하이퍼파라미터(학습률, 옵티마이저, 배치 사이즈 등)의 설정이 천차만별이라, 효율적으로 원하는 결과를 빠르게 도출해내는 것이 어려웠습니다. 이를 해결하기 위해서는 경험이 중요하다고 느꼈습니다. 각 하이퍼파라미터들이 최적화되기 위해서는 어떤 설정이 효과적인지, 일반적으로 사용되는 초기 학습 설정에 대해 더 공부할 필요가 있다고 생각합니다.

WandB(Weights & Biases)와 같은 하이퍼파라미터 최적화, 실험 추적, 모델 학습 과정 모니터링, 데이터셋 및 결과 시각화 도구를 사용하여 실험을 자동화하고 더 빠르게 원하는 결과를 확인하는 방법이 현재로썬 가장 효율적일 것 같다고 느꼈습니다.